# 部署 Paddlenlp 模型到 Sagemaker 上做 Inference

In [1]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import Session

sess = Session()

# AWS IAM role：請確保此 role 有足夠權限訪問 S3, sagemaker，且 Trust relationships 包含 sagemaker.amazonaws.com
role = "arn:aws:iam::404906229455:role/lab_luka_full_access"

# S3 bucket 名稱
bucket = "lab-luka-test"

sagemaker.config INFO - Fetched defaults config from location: /home/ubuntu/.config/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/ubuntu/.config/sagemaker/config.yaml


In [2]:
pt_mnist_model_data = sess.upload_data(
    path="../uie_model.tar.gz", bucket=bucket, key_prefix="local_deploy_example/model_data"
)

print(pt_mnist_model_data)

s3://lab-luka-test/local_deploy_example/model_data/uie_model.tar.gz


## CPU

requirements:
    - paddlenlp
    - paddlepaddle==2.4.2 

```python
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import sagemaker

inference_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region='ap-northeast-1',
    version="1.12",
    py_version="py38",
    instance_type='ml.t2.2xlarge',
    image_scope="inference",
)

pt_mnist_model_data = "s3://lab-luka-test/local_deploy_example/model_data/uie_model.tar.gz"

model = PyTorchModel(
    name='uie-inference-model-cpu2',
    entry_point="inference.py",
    source_dir="src",
    image_uri=inference_image_uri,
    role=role,
    model_data=pt_mnist_model_data,
)

predictor = model.deploy(
    endpoint_name='uie-inference-endpoint-cpu2',
    initial_instance_count=1,
    instance_type="ml.t2.2xlarge",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

```

## GPU

In [37]:
import sagemaker
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region='ap-northeast-1',
    version="1.12",
    py_version="py38",
    instance_type='ml.g4dn.xlarge',
    image_scope="inference",
)
print(inference_image_uri)
# 要設定 runtime 的 python 環境，在實驗時是以 python3.8 做，因此這邊需要找到 python 3.8 的 image，若不設定的話 default python 3.6

763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/pytorch-inference:1.12-cpu-py38


In [41]:
pt_mnist_model_data = "s3://lab-luka-test/local_deploy_example/model_data/uie_model.tar.gz"
model = PyTorchModel(
    name='uie-inference-model-gpu',
    entry_point="inference.py",
    source_dir="src",
    image_uri=inference_image_uri,
    role=role,
    model_data=pt_mnist_model_data,
)
# requirements 必須放在 source_dir 內，他會自己下載

sagemaker.config INFO - Fetched defaults config from location: /home/ubuntu/.config/sagemaker/config.yaml


In [42]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    endpoint_name='uie-inference-endpoint-cpu2',
    initial_instance_count=1,
    instance_type="ml.t2.2xlarge",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

------!

In [45]:
dummy_data = {"inputs": "臺灣臺南地方法院民事判決109年度訴字第1159號原告邱偉翰被告王澤祐上列當事人間請求侵權行為損害賠償事件，原告提起刑事附帶民事訴訟，經本院刑事庭裁定移送前來(109年度附民字第96號)，本院於民國109年11月18日言詞辯論終結，判決如下：主文被告應給付原告新臺幣參萬捌仟貳佰玖拾元，及自民國一○九年三月二十一日起至清償日止，按年息百分之五計算之利息。原告其餘之訴及假執行之聲請均駁回。本判決第一項得假執行。事實及理由一、原告起訴主張：(一)被告前於民國108年10月4日凌晨0時23分許，在臺南市○區○○街00號對面，基於傷害他人身體之犯意，徒手掐住原告之頸部，將原告自機車上拉下，致原告人車倒地，因而受有右手腕扭傷、頸部挫傷等傷害（下稱系爭傷害）。原告因上開傷害，受有如下損失：1、醫療費用合計新臺幣（下同）73,080元：(1)原告因上開傷害需每月定期回診，每次掛號費150元；又每月需進行6次復健治療，每次部分負擔50元、震動或其他機械性治療技術費（物理治療）150元、操作治療技術費150元。(2)原告住家距離診所4.7公里，每次前往回診或復健單趟計程車資195元，每次來回1趟，單月6次。(3)原告每月需使用6包「白芷金絲膏」（10片\/包），每包100元，單月合計600元。(4)以上單月醫療費用合計6,090元，共求償12個月即73,080元。2、精神慰撫金50萬元：原告因上開傷害長期疼痛造成無法入睡及精神折磨，工作服務品質降低、工作時間縮減，須經常調班就醫，騎乘機車時對人群有恐懼陰影，為此請求被告應賠償慰撫金50萬元。(二)爰依侵權行為之法律關係提起本訴。聲明：⒈被告應給付原告573,080元，及自起訴狀繕本送達翌日起至清償日止，按年息百分之五計算之利息；⒉願供擔保，請准宣告假執行。二、被告則以：伊於事發當時認為原告是黑道想對伊不利，為自保才動手，事發隔天就有傳簡訊向原告道歉。對於原告提出之醫療單據沒有爭執，但原告主張之醫療費用應提出單據，伊看原告公開之臉書，原告於108年10月10日就去看國慶煙火，同年11月18日還可以到阿里山，本件傷害事件沒有影響原告的行動及健康，原告還有去幫新人拍照；不同意給付交通費用及慰撫金等語，資為抗辯。聲明：原告之訴及假執行之聲請均駁回。三、得心證之理由：(一)本件被告因於上揭時地傷害原告，業經本院刑事庭以109年度訴字第151號刑事判決認定被告犯傷害罪，並判處拘役50日（得易科罰金）（下稱系爭刑事案件）等情，業經本院職權調閱上開刑事案卷核閱無訛（見訴字卷第117頁，電子卷證見同卷第39頁，下稱系爭刑事案卷）。至於被告辯稱其於事發當時認為原告有意傷害伊，伊係正當防衛云云，惟據原告於系爭刑事案件109年5月12日審理期日以證人身份結證否認有意傷害被告，並陳稱：伊於事發當時接獲被告前妻即訴外人劉依玲電話，請伊前往現場，伊不知劉依玲為何打電話給伊，但因為伊之前是汽車業務，原告與劉依玲之前曾向伊買車，伊猜想可能是發生車禍之類，才騎乘機車前往現場，到現場時伊還有向被告伸手打招呼，被告突然衝過來掐住伊脖子，將伊從機車上拉下來等語；被告則於同上刑事案件審理期日自承：原告於事發當天騎摩托車過來，不是衝向伊，是騎向伊這邊來，騎過來之後有停下來，伊站在機車正前方詢問原告是不是一直在跟蹤伊等語。則據被告自承之內容，原告於事發當時係正常騎乘機車靠近，且有停下來與被告對話，依常情一般人不致誤認存在原告有意攻擊之現在不法侵害，況被告如誤認原告有意攻擊，為避免遭原告騎乘機車蓄意衝撞，正常反應在原告騎乘機車接近時應係反射性跳開，詎被告尚站在機車前方與原告對話，益證被告於事發當時並無誤認原告有意攻擊被告，被告辯稱其傷害原告係出於正當防衛云云，自不可採信。本院審酌上開證據，認為原告主張其於前揭時地遭被告傷害而致傷之事實，真實可信。是以，被告既不法侵害原告成傷，自應就原告所受損害負侵權行為損害賠償責任。(二)按因故意或過失，不法侵害他人之權利者，負損害賠償責任；不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任；又不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額，民法第184條第1項前段、第193條第1項、第195條第1項分別定有明文。茲就原告請求賠償項目及金額是否准許，分述如下：⒈醫療費用部分：查本件據原告所提出全部醫療支出費用單據合計金額僅8,290元如附件附表所示，原告主張其因系爭傷害，經醫師診斷建議每月須復健6次云云，未舉證以實其說（訴字卷第77頁），自應以實際支出費用金額為據。又原告請求被告賠償共12個月之醫療費用云云，然據卷附原告所提出維新診所於109年1月3日所出具診斷證明書，僅記載醫師建議繼續門診追蹤及規律復健治療6個月（附民卷第13頁），原告復未能證明於如附件附表最後一次於109年6月9日復健門診紀錄之後，尚有其他醫療費用支出，應認其得請求被告賠償之醫療費用，以附件附表醫療費用單據合計金額共8,290元為限。⒉交通費用部分：原告請求被告賠償其搭乘計程車往返醫療院所就診所支出之交通費用部分，業據被告表示不同意給付，並抗辯原告應係騎乘機車往返醫療院所，並無所謂計程車車資支出等語。而原告雖主張其狀況嚴重的時候會搭乘計程車等語，然無法特定如附件附表所示就醫紀錄哪幾次係搭乘計程車、哪幾次係自行騎乘機車前往就診，亦無法提出任何搭乘計程車而有支出計程車車資之證據（訴字卷第82頁），應認此部分損失不能證明，其請求被告賠償此部分費用自不能准許。⒊白芷金絲膏部分：原告未能就此部分支出之必要性舉證（訴字卷第77頁），其請求被告賠償此部分支出，亦無法准許。⒋精神慰撫金部分：⑴按被害人受有非財產上損害，請求加害人賠償相當金額之慰撫金時，法院對於慰撫金之量定，應斟酌實際加害情形、所造成之影響、被害人痛苦程度、兩造之身分、地位、經濟情形及其他各種狀況，以核定相當之數額（最高法院51年臺上字第223號、47年臺上字第1221號判例意旨可資參照）。⑵查原告因系爭傷害行為而受有前開身體之傷害，其身心必因而承受相當之痛苦，是原告本於侵權行為之法律關係，請求被告賠償其非財產上之損害，洵屬有據。爰審酌原告於本院審理時自述教育程度為大學畢業、業商，被告則於本院審理時自述教育程度為碩士、職業為教師（見訴字卷第78、76頁），暨雙方依卷附稅務電子閘門財產所得調件明細表所載之資力情形（見訴字卷第51至66頁）等兩造之身分、智識程度、經濟能力、傷害情節、原告所受之傷勢及生理上所受之痛苦等一切情狀，認原告得請求被告賠償之精神慰撫金以3萬元為適當，逾此範圍，不應准許。(三)末按給付無確定期限者，債務人於債權人得請求給付時，經其催告而未為給付，自受催告時起，負遲延責任；其經債權人起訴而送達訴狀，與催告有同一之效力，民法第229條第2項定有明文。又遲延之債務，以支付金錢為標的者，債權人得請求依法定利率計算之遲延利息；但約定利率較高者，仍從其約定利率；應付利息之債務，其利率未經約定，亦無法律可據者，週年利率為百分之5，同法第233條第1項、第203條亦有明定。原告對於被告之侵權行為損害賠償請求權，係屬給付未有確定期限之金錢債權，從而，原告併予請求自起訴狀繕本送達被告之翌日即109年3月21日（見附民卷第25頁送達證書）起至清償日止，按週年利率百分之5計算之利息，亦屬有據，應予准許。四、綜上所述，原告本於侵權行為之法律關係，請求被告賠償醫療費用8,290元、慰撫金3萬元，共38,290元，及自109年3月21日起至清償日止，按週年利率百分之5計算之利息，為有理由，應予准許；逾此部分之請求，為無理由，應予駁回。五、本件原告勝訴部分，所命給付之金額未逾50萬元，依民事訴訟法第389條第1項第5款規定，應依職權宣告假執行；至原告敗訴部分，其假執行之聲請即失所附麗，應予駁回。六、本件事證已臻明確，兩造其餘攻擊防禦方法及所提證據，於判決結果不生影響，爰不一一論列，併此敘明。七、本件損害賠償事件乃刑事附帶民事訴訟，由本院刑事庭移送前來，迄本院言詞辯論終結止，當事人並無任何裁判費或其他訴訟費用之支出，自無諭知訴訟費用負擔之必要，併此指明。中華民國109年12月25日民事第三庭法官許嘉容以上正本證明與原本無異。如不服本判決，應於判決送達後20日內，向本院提出上訴狀並表明上訴理由，如於本判決宣示後送達前提起上訴者，應於判決送達後20日內補提上訴理由書（須附繕本）。如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國109年12月29日書記官謝靜茹"}
dummy_data = {'inputs': "臺灣士林地方法院民事判決109年度訴字第2056號原告張益銘被告施怡伶訴訟代理人李念庭徐邦凱複代理人黃琦閎上列當事人間請求損害賠償事件，原告於本院刑事庭109年度審交簡字第289號過失傷害案件提起刑事附帶民事訴訟，經本院刑事庭以109年度審交附民字第408號裁定移送前來，本院於民國110年4月15日言詞辯論終結，判決如下：主文被告應給付原告新臺幣叁拾貳萬伍仟伍佰肆拾壹元，及自民國一百零九年九月二十六日起至清償日止，按年息百分之五計算之利息。原告其餘之訴駁回。訴訟費用由被告負擔二分之一，餘由原告負擔。本判決第一項得假執行；但被告如以新臺幣叁拾貳萬伍仟伍佰肆拾壹元為原告預供擔保，得免為假執行。原告其餘假執行之聲請駁回。事實及理由壹、程序事項一、按本於道路交通事故有所請求而涉訟者，不問其標的金額或價額一律適用簡易程序。又修正之民事訴訟法簡易訴訟程序，依本施行法第12條第10項公告施行後，於修正前已繫屬之事件，其法院管轄權及審理程序，未經終局裁判者，適用修正後之規定。修正後民事訴訟法第427條第2項第11款及民事訴訟法施行法第4條之1第1款分別定有明文。經查，本件原告於上揭民事訴訟法第427條第2項第11款公告施行前，本於道路交通事故而提起本件刑事附帶民事訴訟請求，經本院刑事庭以裁定移送前來，惟於上揭民事訴訟法第427條第2項第11款公告施行前尚未審結，依上揭法律意旨即應一律適用簡易程序，並經本院於民國110年1月28日言詞辯論期日當庭諭知本件應改變訴訟程序，依簡易訴訟程序審理（見本院卷第42頁），合先敘明。二、按訴狀送達後，原告不得將原訴變更或追加他訴。但擴張或減縮應受判決事項之聲明者、不在此限；原告於判決確定前，得撤回訴之全部或一部。但被告已為本案之言詞辯論者，應得其同意。訴之撤回應以書狀為之。但於期日，得以言詞向法院或受命法官為之。民事訴訟法第255條第1項第3款及第262條第1項前段定有明文。經查，本件原告提起附帶民事訴訟起訴狀上記載原告請求MWU-0825號普通重型機車之損害賠償金額新臺幣（下同）24,720元，此部分請求雖非刑事判決有罪之過失傷害事實之損害而不符合提起刑事附帶民事訴訟要件，惟原告仍於言詞辯論期日表示就此部分請求而為訴之追加，原告並經本院裁定後繳交裁判費1,000元，則原告原請求被告應給付總金額為670,112元。嗣於訴訟進行中，於110年4月15日當庭撤回關於機車損害24,720元之請求，並減縮聲明為：被告應給付原告645,392元，及自起訴狀繕本送達翌日起至清償日止，按週年利率百分之5計算之利息。被告亦當庭對於該撤回表示沒有意見（見本院卷第90至91頁），而生該車損請求部分之訴訟撤回之效力，而上開聲明之變更，係減縮其聲明，於法並無不合，應予准許。貳、實體事項一、原告起訴主張略以：（一）被告於109年5月7日23時20分許，駕駛車牌號碼000-0000號自用小客車（下稱系爭小客車），沿臺北市內湖區星雲街由西往北方向行駛，行經該路段與成功路4段61巷交岔路口欲左轉進入成功路4段61巷時，本應注意遵守道路交通規則轉彎車應讓直行車先行，竟疏未注意貿然左轉，適原告騎乘車牌號碼000-0000號普通重型機車（下稱系爭機車）沿同路段對向行經該處，因煞避不及，致被告之系爭小客車前車頭與原告之系爭機車前車頭發生碰撞（下稱系爭事故），原告因而人車倒地，並受有左膝挫傷、左膝後十字韌帶斷裂、左手肘及左腳踝挫擦傷等傷害（下稱系爭傷害）。（二）系爭事故致原告受有如下之損害：1.醫療費用：原告自系爭事故發生時起，已支出醫藥費用共22,154元。2.不能工作期間之薪資損失：原告於系爭事故發生前，受僱於訴外人信義房屋股份有限公司（下稱信義房屋）擔任業務乙職，每月平均薪資80,856元。惟系爭事故發生後，前後於109年5月11日、同年6月3日、同年8月11日至三軍總醫院附設民眾診療服務處（下稱三軍總醫院），經醫囑分別表示宜休1週、1個月及2個月，故原告於系爭事故發生後至提起本件刑事附帶民事訴訟109年10月7日止，共計153日均無法工作，受有薪資損失412,363元（計算式：80,856÷30×153＝412,363，元以下四捨五入）。3.增加之交通必要費用：系爭事故發生迄今因回診、復健已支出計程車車資10,875元。4.精神慰撫金：系爭事故發生後，尚須持續看診及復健，家中經濟頓失，致其生理及心理承受極大痛苦，且被告於案發後均未向原告致歉或有任何慰問之情，亦未與原告商討和解事宜，犯後態度惡劣，故請求精神慰撫金200,000元。（三）綜上所述，爰依民法第184條第1項前段、第193條第1項、第195條第1項、第213條第1項及第216條第1項前段提起本件訴訟，並聲明：1.被告應給付原告645,392元，及自起訴狀繕本送達翌日起至清償日止，按週年利率百分之5計算之利息。2.原告願供擔保，請准宣告假執行。二、被告抗辯則略以：（一）對於鈞院109年度審交簡字第289號刑事判決認定之犯罪事實並無意見。（二）關於損害賠償之項目抗辯：1.原告所提證物與原告請求22,514元之費用不符。且原告提出國術館之醫療單據，並非經健保醫院所開立，並非應由原告負擔。2.對於原告薪資扣繳憑單年薪為970,266元，每月薪資80,856元，然依原告職業特殊性以觀，一般房仲業務每月固定薪資依市場狀況，其信義房屋每月約落於49,286元，且是否成交房屋對於房仲薪資影響甚鉅，並非每月均能成交，原告亦無法證明其休養期間每月均能成交。又三軍總醫院之診斷證明書記載宜休養共2個月，應係自系爭事故後宜休養2個月，而非原告主張之153天，原告請求之時間過長，故原告得請求之金額應以98,572元為宜（計算式：49,286×2＝98,572）。3.對原告主張增加之交通必要費用，被告並不爭執。4.精神慰撫金：原告請求200,000元之慰撫金過高，應以50,000元為適當。（三）綜上所述，資為抗辯。並聲明：原告之訴駁回。三、本院之判斷（一）原告起訴主張被告於109年5月7日23時20分許，駕駛系爭小客車，沿臺北市內湖區星雲街由西往北方向行駛，行經該路段與成功路4段61巷交岔路口欲左轉進入成功路4段61巷時，本應注意遵守道路交通規則轉彎車應讓直行車先行，竟疏未注意貿然左轉，適原告騎乘系爭機車沿同路段對向行經該處，因煞避不及，致生系爭事故，原告因而人車倒地，並受有系爭傷害等情，為被告所不爭執。並有本院刑事庭109年度審交簡字第289號刑事案卷所附道路交通事故現場圖、道路交通事故調查報告表（一）、（二）、道路交通事故初步分析研判表、談話紀錄表及現場暨車損照片等可按。且被告亦因上揭過失傷害行為，經本院刑事庭以109年度審交簡字第289號刑事簡易判決判處有期徒刑3月，如易科罰金以1,000元折算1日等情，亦有該刑事簡易判決書及刑事案卷在卷可按，堪信為真實。（二）按因故意或過失，不法侵害他人之權利者，負損害賠償責任；又不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任；不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額。民法第184條第1項前段、民法第193條第1項、第195條第1項前段分別定有明定。則被告上揭之過失傷害不法行為造成原告受有系爭傷害，侵害原告身體、健康之權利，當已對原告構成侵權行為，自應負損害賠償責任，而被告自應對於原告負賠償責任，以下分別審究原告各項請求：1.醫療費用部分：⑴原告主張因本件事故所受傷勢，支出醫療費用其中2,954元部分，已提出三軍總醫療費用（骨科）收據共4紙、萬隆力康復健科診所收據共4張為據【見109年度審交附民字第408號卷（下稱附民卷）第29至35頁】，該等單據之真正，為被告所不爭執，上揭收據共8紙，金額共為2,954元，以其就診科別為骨科、復健診所，就診時間均在系爭事故發生後之109年6至同年9月間，上揭原告就診醫療，顯與原告因系爭事故所受有之系爭傷害相關，是原告就此部分請求，應為有理由。⑵原告主張因本件事故所受傷勢，支出見效堂國術館之傳統整復12次費用共計19,200元部分，固提出見效堂國術館證明書及收據各1件（見附民卷第19、21頁）為據，而被告抗辯：原告提出國術館之醫療單據，並非經健保醫院所開立，並非應由原告負擔等情，經查，上揭見效堂國術館證明書及收據，僅能證明原告有此支出，原告並未提出如醫囑、處方簽等足認係治療系爭傷害之必要支出，自難認原告此部分主張為可採。⑶從而，原告得請求被告賠償之醫療費用為2,954元，逾此部分之主張為無理由。2.不能工作期間之薪資損失：原告主張其因系爭車禍受有傷害無法工作之期間為153日，其每月平均薪資為80,856元等情，為被告所否認，並以上揭情詞置辯，經查：⑴原告主張其平均薪資為80,856元一節，業已提出原告108年度各類所得扣繳暨免扣繳憑單（見附民卷第37頁）為據，被告雖抗辯：原告主張薪資過高，因一般房仲業務每月固定薪資，是否成交房屋對於房仲薪資影響甚鉅，並非每月均能成交下，應以其所提出網路新聞（見本院卷第136至138頁）所載原告所任職信義房屋公司平均月薪49,286元為計算基準，惟經本院調取原告近3年之所得資料顯示，原告106年、107年及108年三間均任職信義房屋公司之所得而言，108年度係3年中最低，且原告106年、107年之年度所得均超過1,000,000元，足見原告主張依車禍前一年度之年度所得平均計算其月薪甚為合理。而被告僅憑網路新聞資料中關於任職信義房屋員工之平均月薪，不論資歷及工作成況，而以為認定原告之月薪，尚非可採。⑵原告主張其因系爭車禍受有傷害無法工作之期間為153日，惟經本院向三軍總醫院函詢原告因系爭事故受有系爭傷害後，宜休養多少時間，始得恢復其從事之房屋仲介業務員之工作？經三軍總醫院函覆（略以）：張員（即原告）於109年6月3日就診，理學檢查佐證可能存在十字韌帶損傷，因此安排進一步檢查以確立診斷，故兩者（指109年6月3日診斷疑似十字韌帶損傷與109年7月28日診斷受有左膝後十字韌帶斷裂傷害）為同一傷害；左膝挫傷後造成後十字韌帶損傷斷裂，急性期需1至3月疼痛會減緩，但造成韌帶鬆弛是永久性傷害。建議約需2個月後較合適恢復其工作，然而後十字韌帶斷裂後如仍然無法恢復膝蓋穩定程度，需要接受重建手術治療，因此目前暫無法判定休養時程及恢復工作時間，需持續追蹤等情（見本院卷第70、72頁）。是本院參酌上開回函以及被告所為之抗辯意見，認原告所請求其因系爭車禍受有傷害無法工作之期間應自系爭事故發生時起為2個月為適當。⑶綜上，原告請求因系爭事故而不能工作期間2個月之薪資損失為161,712元（計算式：80,856×2＝161,712）為有理由，而逾此部分之主張為無理由。3.增加之交通必要費用：原告主張系爭事故發生迄今因回診、復健已支出計程車車資10,875元部分，為被告所不爭執，並有原告所提出之車資費用收據為證（見附民卷第41至59頁），且以原告所受系爭傷害及原告所提出三總診斷證明書確實記載建議使用膝關節輔具之情形，確實會影響其行動之便利，是原告就診確有必要搭乘計程車往返，故原告此部分請求為有理由。4.精神慰撫金：按不法侵害他人之人格權，被害人受有非財產上損害，請求加害人賠償相當金額之慰撫金時，法院對於慰撫金之量定，應斟酌實際加害情形、所造成之影響、被害人痛苦之程度、兩造之身分地位經濟情形及其他各種狀況，以核定相當之數額（最高法院47年度台上字第1221號、51年度台上字第223號判例意旨參照）。查原告因被告之過失行為受有系爭傷害，自受有相當之精神及身心上之痛苦，其據以向被告請求賠償精神慰撫金，自屬有據。又原告係大學畢業，從事房屋仲介工作，月薪約100,000元，名下無財產，離婚帶有4歲及9歲兩個小孩，父親罹患口腔癌，其與小孩及父母同住；被告係大學畢業，從事媒體業，家庭狀況小康等情，業據兩造於言詞辯論期日陳述在卷（見本院卷第93頁），而兩造106年度至108年度各類所得及財產狀況，復經本院依職權調取稅務電子閘門財產所得調件明細表附於限制閱覽卷內可參。本院審酌上情暨被告所為侵權行為態樣、主觀上過失及客觀上可歸責程度及原告因系爭事故所受有左膝後十字韌帶斷裂所來之疼痛、對於行動所造成之困難與影響及接受治療過程之辛苦且造成韌帶鬆弛是永久性傷害，而對於原告因而所受精神上之痛苦程度等一切情狀，認原告請求被告給付非財產上之損害200,000元，核屬過高，應以150,000元為相當，逾此部分之請求，礙難准許。（三）按給付無確定期限者，債務人於債權人得請求給付時，經其催告而未為給付，自受催告時起，負遲延責任。其經債權人起訴而送達訴狀，或依督促程序送達支付命令，或為其他相類之行為者，與催告有同一之效力；遲延之債務，以支付金錢為標的者，債權人得請求依法定利率計算之遲延利息；應付利息之債務，其利率未經約定，亦無法律可據者，週年利率為5％，民法第229條第2項、第233條第1項前段及第203條分別定有明文。本件原告依侵權行為法律關係請求被告負損害賠償責任，為無確定期限、無從另為約定利率之債務，是原告自得依上開規定請求被告給付原告325,541元（計算式：2,954＋161,712＋10,875＋150,000＝325,541），及自起訴狀繕本送達翌日即109年9月26日起（見附民卷第3頁刑事附帶民事起訴狀上被告之收受繕本簽名及日期）至清償日止，按週年利率百分之5計算之利息，為有理由，應予准許。四、綜上所述，原告基於侵權行為法律關係，請求被告給付原告325,541元，及自109年9月26日起至清償日止，按週年利率百分之5計算之利息，為有理由，應予准許。逾此部分之請求，則無理由，應予駁回。五、本判決原告勝訴部分，所命給付之價額未逾500,000元，依民事訴訟法第389條第1項第5款規定，應依職權宣告假執行。原告就勝訴部分陳明願供擔保聲請宣告假執行，僅係促使本院之職權發動，無庸另為准駁之諭知。並依職權宣告被告得為原告供擔保，免為假執行。至原告敗訴部分，其假執行之聲請失所附麗，應予駁回。六、本件事證已臻明確，兩造其餘攻擊防禦方法及證據，核與判決結果不生影響，爰不一一論列，併此敘明。七、據上論結，原告之訴為一部有理由、一部無理由，依民事訴訟法第79條第2項、第389條第1項第5款、第390條第2項，判決如主文。中華民國110年4月29日民事第二庭法官方鴻愷以上正本係照原本作成。如對本判決上訴，應於判決送達後20日內向本院提出上訴狀，若委任律師提起上訴者，應一併繳納上訴審裁判費，否則本院得不命補正逕行駁回上訴。中華民國110年4月29日書記官陳紀元。"}

In [46]:
res = predictor.predict(dummy_data)
print(res)

[{'精神慰撫金額': [{'text': '150,000', 'start': 5507, 'end': 5514, 'probability': 0.9481814888050799}, {'text': '150,000元', 'start': 5188, 'end': 5196, 'probability': 0.9977151884524105}], '醫療費用': [{'text': '2,954元', 'start': 3260, 'end': 3266, 'probability': 0.9512461566393569}, {'text': '2,954元', 'start': 3564, 'end': 3570, 'probability': 0.984995948182771}, {'text': '2,954', 'start': 5486, 'end': 5491, 'probability': 0.44946825672604973}], '薪資收入': [{'text': '80,856', 'start': 4452, 'end': 4458, 'probability': 0.9853195344127954}, {'text': '100,000元', 'start': 4883, 'end': 4891, 'probability': 0.9923144819152121}]}]
